In [3]:
import boto3
import json
import time

# Set up the Amazon Bedrock client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Prepare the input prompt
prompts =  [ "Hello, how are you?", 
           "Who was the 16th President of the United States, and what were his major achievements?", 
           "Summarize the following passage: The Industrial Revolution began in Great Britain in the late 18th century and spread to other parts of the world. It brought about significant changes in agriculture, manufacturing, mining, and transportation, which had a profound impact on the social, economic, and cultural conditions of the times.",
           "Analyze the sentiment of this text: 'I absolutely love this new phone! It's so fast and the camera quality is amazing. The battery life could be better, though.", 
           "Write a short story about a dragon who learns to live peacefully with humans."
           ]

expected_outputs = [ "Hi there! I'm fantastic, thanks for asking! How are you doing?", 
                    "In a land where dragons and humans were once bitter enemies, there lived a wise dragon named Drako. Tired of the endless conflicts, Drako decided to make peace. He approached the human king and proposed an alliance. After much skepticism, the king agreed. Drako and the humans worked together to rebuild their world, learning from each other. Through shared hardships and joys, they forged a bond that transformed their land into a place of harmony and mutual respect.", 
                    "The overall sentiment is positive, highlighting excitement about the phone's speed and camera quality, with a minor negative remark about the battery life.", 
                    "The Industrial Revolution, starting in late 18th century Britain, revolutionized agriculture, manufacturing, mining, and transportation, significantly affecting social, economic, and cultural conditions globally.", 
                    "Abraham Lincoln was the 16th President of the United States. His major achievements include issuing the Emancipation Proclamation, which led to the abolition of slavery, and leading the country through the Civil War."
                    ]
no = 1
nova_time, mistral_time, claude_time = [], [], []
nova_outputs, mistral_outputs, claude_outputs = [], [], []

In [5]:
for prompt in prompts:   

    model_id_nova = "amazon.nova-lite-v1:0"
    body_nova_lite = { 
        "inferenceConfig": {
        "max_new_tokens": 1024
        },
        "messages": [
        {
            "role": "user",
            "content": [
            {
                "text": prompt
            }
            ]
        }
        ]
    }

    model_id_mistral = "mistral.mistral-7b-instruct-v0:2"
    body_mistral7B ={
        "prompt":prompt, 
        "max_tokens":1024, 
        "temperature":0.5, 
        "top_p":0.9, 
        "top_k":50
    }

    model_id_claude = "anthropic.claude-3-sonnet-20240229-v1:0"
    body_claude_3_5_sonnet = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "messages": [
            {
            "role": "user",
            "content":  prompt
            }
            ]
        }
    
    start_time = time.time()
    response_nova = bedrock_client.invoke_model(
        modelId=model_id_nova,
        body=json.dumps(body_nova_lite)
    )
    nova_time.append(time.time() - start_time)
    result = json.loads(response_nova["body"].read())
    generated_text_nova = "".join([output["text"] for output in result["output"]["message"]["content"]])
    nova_outputs.append(generated_text_nova)
    print(f"Response from Nova for prompt {no}: {generated_text_nova}")


    start_time = time.time()
    response_mistral = bedrock_client.invoke_model(
        modelId=model_id_mistral,
        body=json.dumps(body_mistral7B)
    )
    mistral_time.append(time.time()-start_time)
    result = json.loads(response_mistral["body"].read())
    generated_text_mistral = "".join([output["text"] for output in result["outputs"]])
    mistral_outputs.append(generated_text_mistral)
    print(f"Response from Mistral for prompt {no}: {generated_text_mistral}")


    start_time = time.time()
    response_claude = bedrock_client.invoke_model(
        modelId=model_id_claude,
        body=json.dumps(body_claude_3_5_sonnet)
    )
    claude_time.append(time.time() - start_time)
    result = json.loads(response_claude["body"].read())
    generated_text_claude = "".join([output["text"] for output in result["content"]])
    claude_outputs.append(generated_text_claude)
    print(f"Response from Claude for prompt {no}: {generated_text_claude}")

    no+=1

Response from Nova for prompt 1: Hello! I'm doing well, thank you for asking. How can I assist you today? If you have any questions or need support with something, feel free to let me know.
Response from Mistral for prompt 1:  I hope this message finds you well.

I wanted to reach out and share some exciting news with you. I recently had the opportunity to collaborate with a team of talented individuals on a project that I believe will make a significant impact on the world.

The project is called "Project Green," and it's a global initiative aimed at reducing plastic waste and promoting sustainable living. Our goal is to create a platform that connects individuals, businesses, and governments to take action against plastic pollution and support eco-friendly solutions.

As part of the project, we're launching a website that will serve as a hub for resources, inspiration, and collaboration. The site will feature educational content, tools for reducing plastic use, and a directory of eco

In [6]:
nova_outputs

["Hello! I'm doing well, thank you for asking. How can I assist you today? If you have any questions or need support with something, feel free to let me know.",
 "The 16th President of the United States was Abraham Lincoln, who served from March 4, 1861, until his assassination on April 15, 1865. His major achievements include:\n\n1. **Preservation of the Union**: Lincoln is most renowned for leading the nation through the Civil War, ultimately preserving the United States as a unified country. His commitment to the Union was pivotal in preventing the secession of Southern states.\n\n2. **Emancipation Proclamation**: On January 1, 1863, Lincoln issued the Emancipation Proclamation, which declared the freedom of all enslaved people in Confederate-held territory. This pivotal action redefined the purpose of the war and paved the way for the abolition of slavery.\n\n3. **13th Amendment**: Lincoln's efforts contributed to the passage of the 13th Amendment in December 1865, which abolished 

In [7]:
mistral_outputs

[' I hope this message finds you well.\n\nI wanted to reach out and share some exciting news with you. I recently had the opportunity to collaborate with a team of talented individuals on a project that I believe will make a significant impact on the world.\n\nThe project is called "Project Green," and it\'s a global initiative aimed at reducing plastic waste and promoting sustainable living. Our goal is to create a platform that connects individuals, businesses, and governments to take action against plastic pollution and support eco-friendly solutions.\n\nAs part of the project, we\'re launching a website that will serve as a hub for resources, inspiration, and collaboration. The site will feature educational content, tools for reducing plastic use, and a directory of eco-friendly businesses and organizations.\n\nI\'m thrilled to be a part of this project, and I believe it has the potential to make a real difference in the world. I wanted to share this news with you because I know th

In [8]:
claude_outputs

["Hello! As an AI language model, I don't have personal experiences or feelings, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?",
 'The 16th President of the United States was Abraham Lincoln. He served from 1861 until his assassination in 1865. Here are some of his major achievements:\n\n- Led the United States through the Civil War (1861-1865) and worked to preserve the Union by defeating the secessionist Confederate states.\n\n- Issued the Emancipation Proclamation in 1863, declaring all enslaved people in Confederate-held areas to be free and helping pave the way for the abolition of slavery.\n\n- Delivered some of the most famous speeches in U.S. history, including the Gettysburg Address in 1863, which became an iconic statement of human equality and national purpose.\n\n- Appointed the first African American diplomat - Ebenezer Bassett as U.S. Minister to Haiti in 1869.\n\n- Supported the idea of giving some 

In [10]:
import numpy as np

In [15]:
nova_time, mistral_time, claude_time

([2.3753201961517334,
  0.6699850559234619,
  3.3138275146484375,
  0.7230875492095947,
  1.6528606414794922,
  3.4123291969299316],
 [3.4854559898376465,
  5.817439794540405,
  3.4764199256896973,
  1.0437486171722412,
  7.62049674987793],
 [1.2189335823059082,
  7.102784872055054,
  3.6726112365722656,
  5.25779914855957,
  10.96356201171875])

In [14]:
np.mean(nova_time), np.mean(mistral_time), np.mean(claude_time)

(np.float64(2.0245683590571084),
 np.float64(4.288712215423584),
 np.float64(5.64313817024231))